In [1]:
import sys 
import networkx as nx
import pandas as pd
import numpy as np
import pickle as pic
import random

import cassiopeia.TreeSolver.simulation_tools.simulation_utils as sim_utils
import cassiopeia.TreeSolver.simulation_tools.dataset_generation as data_gen
from cassiopeia.TreeSolver.Node import Node
from cassiopeia.TreeSolver.Cassiopeia_Tree import Cassiopeia_Tree

from tqdm import tqdm_notebook
import matplotlib.pyplot as plt

import subprocess

#import seaborn as sns
import os

In [2]:
#Recursive function to travel up the tree and grab a missing character from a node's first ancester
#where it is no longer missing
def get_predecessor_val(tree, node, ind):
    for pred in tree.network.predecessors(node):
        if pred.char_vec[ind] == '-' or pred.char_vec[ind] == '*':
            return get_predecessor_val(tree, pred, ind)
        else:
            return pred.char_vec[ind]

In [50]:
#Specify number of cells and the imputation algorithm used
NUM_CELLS = 400

run = []
accuracies = []
h_accuracies = []
s_accuracies = []
recon = []

# Iterate over the heritable dropout percentages
for recon_method in ["avg", "lookahead2", "knn", "avg_h", "lookahead_h2", "knn_h"]:
    print(recon_method)

#     for drop in range(6):

#         hdropout_percent = str(drop)
#         print("Percent: " + hdropout_percent)

    #Establish path
    path = "/data/yosef2/users/richardz/projects/cont_birthdeath/" + str(NUM_CELLS) + "cellsepisilence/" #+ hdropout_percent + "percent"

    #Main loop

    for num in range(0, 50):
        #Load the tree, the character matrix, and post-process the tree
        test_recon = pic.load(open(path + "/" + recon_method + "/dropout_cm" + str(num) + "_" + recon_method + ".pkl", 'rb'))
        dropout_cm = pd.read_csv(path + '/dropout_cm' + str(num) + '.txt', sep = '\t', index_col = 0)
        test_ppg = test_recon.post_process(dropout_cm)

        #Create a dictionary to map the names of the nodes to their memory addresses
        names_to_leaves = {}
        for i in list(test_ppg.network.nodes):
            if i.name != "state-node":
                names_to_leaves[i.name] = i

        #For each named node in the character matrix (post dropout), grab its location in memory and go through
        #its character vector. For each missing value, grab the character from its first non-missing ancestor in 
        #the reconstructed tree. If a character value is still missing at root, implicitly assume its value is 0.
        #Then add the reconstructed row to the list and construct a data frame.
        reconstructed_rows = []
        for index, row in dropout_cm.iterrows():
            node = names_to_leaves[index]
            reconstructed_row = []
            for i in range(len(row)):
                if row[i] == "-" or row[i] == "*":
                    char = get_predecessor_val(test_ppg, node, i)
                    if char == None:
                        char = '0'
                    reconstructed_row.append(char)
                else:
                    reconstructed_row.append(row[i])
            reconstructed_rows.append(reconstructed_row)
        reconstructed_cm = pd.DataFrame(reconstructed_rows)

        #Read in the ground-truth character matrix, before dropout is introduced
        ground_truth_cm = pd.read_csv(path + "/ground_truth_cm" + str(num) + ".txt", sep='\t', index_col = 0)

        #Go through the post-dropout character matrix. For each missing character in the post-dropout character matrix,
        #check to see if the imputed character matrix matches the character in the ground-truth character matrix
        #at that character. If yes, it is considered imputed correctly. Keep a running total of the number of total
        #dropouts and the number of imputed correctly to create a proportion correct.
        num_dropped_h = 0
        num_dropped_s = 0
        num_correct_h = 0
        num_correct_s = 0
        for k in range(dropout_cm.shape[0]):
            for j in range(dropout_cm.shape[1]):
                if dropout_cm.iloc[k,j] == "-":
                    num_dropped_s += 1
                    if str(ground_truth_cm.iloc[k,j]) == str(reconstructed_cm.iloc[k,j]):
                        num_correct_s += 1
                if dropout_cm.iloc[k,j] == "*":
                    num_dropped_h += 1
                    if str(ground_truth_cm.iloc[k,j]) == str(reconstructed_cm.iloc[k,j]):
                        num_correct_h += 1

        accuracy = (num_correct_s + num_correct_h)/(num_dropped_s + num_dropped_h)
        accuracies.append(accuracy)
        h_accuracy = num_correct_h/num_dropped_h
        h_accuracies.append(h_accuracy)
        s_accuracy = num_correct_s/num_dropped_s
        s_accuracies.append(s_accuracy)
        run.append(num)
        recon.append(recon_method)
        print(num, accuracy)
        
#         #Write the proportions correctly imputed to CSV
#         import csv

#         with open('/data/yosef2/users/richardz/projects/dropout_testing/heritable_testing/' + str(NUM_CELLS) + 'cells/' + recon_method + '_split_on_h_accuracies' + hdropout_percent + '.csv', 'w') as csvFile:
#             writer = csv.writer(csvFile)
#             writer.writerow(accuracies)
#         csvFile.close()

out = pd.DataFrame([run, accuracies, h_accuracies, s_accuracies, recon])
out = out.T
out = out.rename(columns = {0: 'Run', 1: 'Total', 2: 'Heritable', 3: 'Stochastic', 4: 'Type'})

out.to_csv("/data/yosef2/users/richardz/projects/cont_birthdeath/" + str(NUM_CELLS) + "cellsepisilence_dropout_prop_correct.txt", sep = "\t", index = False)

avg
0 0.8593492277702804
1 0.8530076888285844
2 0.6886943836615609
3 0.7548857693366364
4 0.9286876756916703
5 0.7846252607928101
6 0.7491971740526654
7 0.8206801107156979
8 0.8592283628779979
9 0.8063883989809916
10 0.7336276674025018
11 0.7824275362318841
12 0.7845429452833252
13 0.8559185859667916
14 0.9013312451057165
15 0.7577160493827161
16 0.7805212620027435
17 0.8286845730027548
18 0.8372605759955905
19 0.8074756784434204
20 0.8537624926513815
21 0.7892838742916023
22 0.8167737683308618
23 0.7800055694792537
24 0.8167821401077752
25 0.870026525198939
26 0.7459939080916435
27 0.80844422539839
28 0.7415275670207385
29 0.8048324656254172
30 0.8404634581105169
31 0.7900757209602062
32 0.748013090229079
33 0.843436754176611
34 0.8690161527165933
35 0.8256216556499842
36 0.7978003384094755
37 0.8468199886428166
38 0.745221971407073
39 0.7938573659552316
40 0.8519292380554395
41 0.8690932311621967
42 0.7786446382132477
43 0.6206939281288724
44 0.7658284287497771
45 0.7841144362883493


In [44]:
recon_method = "avg"

path = "/data/yosef2/users/richardz/projects/cont_birthdeath/" + str(NUM_CELLS) + "cells2/" #+ hdropout_percent + "percent"
accuracies = []

num = 3

test_recon = pic.load(open(path + recon_method + "/dropout_cm" + str(num) + "_" + recon_method + ".pkl", 'rb'))
dropout_cm = pd.read_csv(path + 'dropout_cm' + str(num) + '.txt', sep = '\t', index_col = 0)
test_ppg = test_recon.post_process(dropout_cm)

names_to_leaves = {}

for i in list(test_ppg.network.nodes):
    if i.name != "state-node":
        names_to_leaves[i.name] = i

reconstructed_rows = []
for index, row in dropout_cm.iterrows():
    print(index)
    node = names_to_leaves[index]
    reconstructed_row = []
    for i in range(len(row)):
        if row[i] == '-' or row[i] == '*':
            char = get_predecessor_val(test_ppg, node, i)
            if char == None:
                char = '0'
            reconstructed_row.append(char)
        else:
            reconstructed_row.append(node.char_vec[i])
    reconstructed_rows.append(reconstructed_row)
reconstructed_cm = pd.DataFrame(reconstructed_rows)

ground_truth_cm = pd.read_csv(path + "ground_truth_cm" + str(num) + ".txt", sep='\t', index_col = 0)

num_dropped = 0
num_correct = 0
for k in range(400):
    for j in range(30):
        print("new")
        print(dropout_cm.iloc[k,j])
        if dropout_cm.iloc[k,j] == "-" or dropout_cm.iloc[k,j] == "*":
            num_dropped += 1
            print(ground_truth_cm.iloc[k,j])
            print(reconstructed_cm.iloc[k,j])
            if str(ground_truth_cm.iloc[k,j]) == str(reconstructed_cm.iloc[k,j]):
                num_correct += 1
                print("match")

print(dropout_cm.shape)
print(ground_truth_cm.shape)
print(reconstructed_cm.shape)
accuracy = num_correct/num_dropped
accuracies.append(accuracy)
print(num, accuracy)


c0
c1
c2
c3
c4
c5
c6
c7
c8
c9
c10
c11
c12
c13
c14
c15
c16
c17
c18
c19
c20
c21
c22
c23
c24
c25
c26
c27
c28
c29
c30
c31
c32
c33
c34
c35
c36
c37
c38
c39
c40
c41
c42
c43
c44
c45
c46
c47
c48
c49
c50
c51
c52
c53
c54
c55
c56
c57
c58
c59
c60
c61
c62
c63
c64
c65
c66
c67
c68
c69
c70
c71
c72
c73
c74
c75
c76
c77
c78
c79
c80
c81
c82
c83
c84
c85
c86
c87
c88
c89
c90
c91
c92
c93
c94
c95
c96
c97
c98
c99
c100
c101
c102
c103
c104
c105
c106
c107
c108
c109
c110
c111
c112
c113
c114
c115
c116
c117
c118
c119
c120
c121
c122
c123
c124
c125
c126
c127
c128
c129
c130
c131
c132
c133
c134
c135
c136
c137
c138
c139
c140
c141
c142
c143
c144
c145
c146
c147
c148
c149
c150
c151
c152
c153
c154
c155
c156
c157
c158
c159
c160
c161
c162
c163
c164
c165
c166
c167
c168
c169
c170
c171
c172
c173
c174
c175
c176
c177
c178
c179
c180
c181
c182
c183
c184
c185
c186
c187
c188
c189
c190
c191
c192
c193
c194
c195
c196
c197
c198
c199
c200
c201
c202
c203
c204
c205
c206
c207
c208
c209
c210
c211
c212
c213
c214
c215
c216
c217
c218
c219
c220
c221


new
0
new
396
new
0
new
407
new
495
new
267
new
409
new
0
new
-
0
0
match
new
-
0
377
new
-
316
0
new
0
new
0
new
473
new
-
0
0
match
new
-
0
0
match
new
-
494
0
new
362
new
0
new
366
new
462
new
338
new
0
new
0
new
0
new
0
new
482
new
479
new
499
new
-
162
162
match
new
-
0
0
match
new
-
0
0
match
new
0
new
303
new
0
new
-
490
0
new
-
0
0
match
new
-
0
0
match
new
0
new
465
new
453
new
-
0
0
match
new
-
287
287
match
new
-
0
0
match
new
-
0
0
match
new
-
318
318
match
new
-
0
0
match
new
336
new
0
new
0
new
*
129
0
new
*
0
0
match
new
*
493
0
new
450
new
0
new
0
new
-
482
482
match
new
-
479
479
match
new
-
0
0
match
new
162
new
0
new
287
new
209
new
303
new
417
new
0
new
0
new
496
new
302
new
431
new
453
new
-
237
0
new
-
244
0
new
-
0
437
new
377
new
0
new
309
new
0
new
0
new
410
new
180
new
0
new
0
new
0
new
0
new
0
new
0
new
404
new
412
new
162
new
0
new
396
new
0
new
0
new
0
new
478
new
419
new
0
new
481
new
376
new
466
new
373
new
0
new
498
new
0
new
0
new
0
new
0
new
0
new
426


291
291
match
new
-
266
266
match
new
0
new
0
new
0
new
0
new
0
new
0
new
300
new
378
new
0
new
-
39
39
match
new
-
479
479
match
new
-
0
0
match
new
0
new
0
new
386
new
0
new
0
new
188
new
0
new
413
new
0
new
357
new
0
new
453
new
392
new
0
new
0
new
-
0
0
match
new
*
372
0
new
*
471
0
new
456
new
0
new
257
new
-
0
0
match
new
-
384
384
match
new
-
0
0
match
new
0
new
0
new
0
new
-
479
479
match
new
-
404
404
match
new
-
65
65
match
new
288
new
0
new
396
new
-
0
0
match
new
-
0
0
match
new
-
314
314
match
new
59
new
0
new
0
new
0
new
0
new
317
new
248
new
0
new
0
new
478
new
134
new
0
new
0
new
0
new
0
new
462
new
0
new
363
new
371
new
461
new
0
new
482
new
479
new
499
new
162
new
0
new
0
new
0
new
303
new
0
new
0
new
0
new
0
new
-
0
0
match
new
-
465
465
match
new
-
453
453
match
new
0
new
228
new
0
new
-
0
0
match
new
-
318
318
match
new
-
0
0
match
new
336
new
0
new
161
new
*
129
0
new
*
0
0
match
new
*
493
0
new
300
new
0
new
386
new
0
new
479
new
0
new
99
new
0
new
0
new
0
new
0


93
new
300
new
0
new
0
new
0
new
479
new
0
new
-
0
0
match
new
-
0
0
match
new
-
0
0
match
new
0
new
173
new
0
new
*
74
0
new
*
413
413
match
new
*
390
0
new
-
357
357
match
new
-
0
470
new
-
453
453
match
new
0
new
0
new
0
new
0
new
*
372
0
new
*
471
0
new
456
new
249
new
257
new
-
461
0
new
-
384
384
match
new
-
0
316
new
0
new
0
new
0
new
482
new
479
new
499
new
-
162
162
match
new
-
0
0
match
new
-
0
0
match
new
0
new
303
new
0
new
490
new
0
new
0
new
0
new
465
new
453
new
0
new
287
new
0
new
0
new
318
new
0
new
336
new
0
new
0
new
*
129
0
new
*
0
0
match
new
*
493
0
new
-
300
300
match
new
-
293
0
new
-
0
303
new
0
new
479
new
448
new
228
new
0
new
0
new
0
new
343
new
0
new
*
74
0
new
*
413
413
match
new
*
390
0
new
357
new
0
new
453
new
0
new
0
new
419
new
-
0
0
match
new
*
372
0
new
*
471
0
new
456
new
249
new
257
new
0
new
384
new
0
new
-
300
300
match
new
-
450
0
new
-
0
0
match
new
0
new
479
new
456
new
345
new
0
new
0
new
0
new
0
new
270
new
0
new
413
new
0
new
357
new
304
n

0
new
456
new
333
new
257
new
0
new
384
new
0
new
89
new
0
new
0
new
482
new
479
new
0
new
162
new
290
new
287
new
209
new
303
new
0
new
0
new
0
new
0
new
467
new
0
new
453
new
0
new
0
new
0
new
0
new
0
new
421
new
0
new
315
new
0
new
180
new
0
new
480
new
-
0
0
match
new
-
0
0
match
new
-
0
0
match
new
479
new
404
new
76
new
0
new
0
new
396
new
0
new
0
new
0
new
267
new
0
new
463
new
0
new
286
new
330
new
462
new
0
new
0
new
121
new
0
new
0
new
0
new
408
new
0
new
462
new
0
new
265
new
-
300
300
match
new
-
453
453
match
new
-
0
0
match
new
0
new
479
new
0
new
99
new
0
new
0
new
0
new
0
new
377
new
0
new
413
new
0
new
357
new
483
new
453
new
0
new
369
new
0
new
0
new
*
372
0
new
*
471
0
new
456
new
0
new
257
new
0
new
384
new
0
new
371
new
461
new
0
new
482
new
479
new
499
new
162
new
410
new
0
new
90
new
303
new
0
new
0
new
0
new
0
new
0
new
465
new
453
new
405
new
228
new
0
new
413
new
318
new
0
new
336
new
0
new
161
new
*
129
0
new
*
0
0
match
new
*
493
0
new
371
new
461
new
0
new


...

-
451
451
match
new
0
new
0
new
0
new
0
new
404
new
412
new
0
new
396
new
396
new
0
new
0
new
0
new
0
new
419
new
257
new
481
new
376
new
0
new
-
373
373
match
new
-
0
0
match
new
-
498
498
match
new
0
new
0
new
169
new
-
0
0
match
new
-
0
0
match
new
-
426
426
match
new
0
new
354
new
405
new
0
new
137
new
429
new
479
new
404
new
0
new
0
new
0
new
396
new
-
0
0
match
new
-
0
0
match
new
-
0
0
match
new
267
new
0
new
0
new
0
new
0
new
494
new
131
new
0
new
223
new
0
new
0
new
0
new
*
491
248
new
*
331
0
new
0
new
462
new
0
new
0
new
*
344
0
new
*
0
0
match
new
*
149
0
new
0
new
404
new
163
new
0
new
190
new
396
new
-
68
68
match
new
-
358
0
new
-
0
0
match
new
0
new
0
new
187
new
-
483
483
match
new
-
0
0
match
new
-
0
260
new
372
new
0
new
428
new
0
new
329
new
274
new
331
new
0
new
0
new
*
242
0
new
*
495
419
new
451
new
371
new
0
new
189
new
482
new
479
new
499
new
162
new
0
new
336
new
0
new
303
new
0
new
0
new
0
new
0
new
0
new
465
new
453
new
-
0
0
match
new
-
228
228
match
new
-


new
131
new
0
new
223
new
0
new
0
new
0
new
*
491
248
new
*
331
0
new
0
new
462
new
206
new
0
new
-
182
0
new
-
0
0
match
new
-
0
498
new
208
new
479
new
0
new
0
new
0
new
0
new
-
396
0
new
-
0
0
match
new
-
70
70
match
new
123
new
0
new
186
new
462
new
0
new
453
new
375
new
393
new
0
new
136
new
0
new
475
new
0
new
347
new
0
new
-
113
113
match
new
-
435
0
new
-
480
480
match
new
0
new
0
new
178
new
479
new
404
new
115
new
479
new
0
new
396
new
0
new
0
new
0
new
267
new
460
new
0
new
0
new
0
new
323
new
*
436
0
new
*
65
0
new
0
new
0
new
0
new
471
new
-
0
0
match
new
-
0
0
match
new
-
0
0
match
new
462
new
397
new
0
new
0
new
0
new
0
new
-
479
479
match
new
-
404
404
match
new
-
65
65
match
new
-
288
288
match
new
-
0
0
match
new
-
396
396
match
new
0
new
0
new
314
new
59
new
0
new
0
new
0
new
0
new
317
new
0
new
0
new
0
new
478
new
134
new
0
new
0
new
0
new
0
new
462
new
0
new
363
new
0
new
0
new
0
new
-
208
208
match
new
-
479
479
match
new
-
0
0
match
new
420
new
0
new
0
new
449
ne

304
new
453
new
469
new
0
new
432
new
325
new
*
372
0
new
*
471
0
new
456
new
0
new
257
new
0
new
384
new
93
new
*
344
0
new
*
0
0
match
new
*
149
0
new
329
new
404
new
163
new
-
0
0
match
new
-
388
388
match
new
-
396
396
match
new
68
new
0
new
0
new
0
new
0
new
187
new
483
new
262
new
0
new
446
new
0
new
428
new
0
new
0
new
274
new
-
331
331
match
new
-
279
0
new
-
0
486
new
*
242
0
new
*
495
419
new
-
451
451
match
new
*
472
0
new
*
394
137
new
0
new
479
new
404
new
484
new
479
new
0
new
396
new
0
new
0
new
0
new
-
267
267
match
new
-
474
474
match
new
-
0
0
match
new
-
110
110
match
new
-
0
0
match
new
-
0
0
match
new
0
new
0
new
0
new
170
new
0
new
0
new
248
new
0
new
0
new
-
462
462
match
new
-
0
0
match
new
-
444
444
match
new
300
new
431
new
0
new
399
new
479
new
408
new
0
new
0
new
0
new
171
new
302
new
0
new
-
443
443
match
new
-
413
413
match
new
-
471
471
match
new
-
357
357
match
new
-
0
0
match
new
-
453
453
match
new
246
new
0
new
0
new
0
new
*
372
0
new
*
471
0
new
456


match
new
0
new
0
new
485
new
482
new
0
new
244
new
-
146
371
new
-
479
479
match
new
-
351
0
new
0
new
214
new
0
new
0
new
0
new
338
new
207
new
405
new
0
new
283
new
0
new
453
new
390
new
0
new
0
new
0
new
387
new
0
new
0
new
158
new
0
new
0
new
152
new
185
new
0
new
256
new
0
new
-
479
479
match
new
-
404
404
match
new
-
0
358
new
0
new
0
new
396
new
0
new
0
new
495
new
267
new
409
new
0
new
0
new
377
new
0
new
0
new
483
new
0
new
-
435
0
new
-
0
0
match
new
-
0
0
match
new
0
new
0
new
0
new
462
new
0
new
489
new
450
new
*
497
0
new
*
328
0
new
482
new
479
new
444
new
162
new
322
new
287
new
209
new
303
new
417
new
0
new
459
new
0
new
0
new
0
new
453
new
0
new
0
new
437
new
0
new
0
new
210
new
0
new
451
new
0
new
180
new
0
new
0
new
0
new
0
new
152
new
482
new
479
new
499
new
-
162
162
match
new
-
0
0
match
new
-
0
0
match
new
-
0
0
match
new
-
303
303
match
new
-
500
0
new
0
new
0
new
0
new
0
new
465
new
453
new
406
new
471
new
492
new
-
0
0
match
new
-
318
318
match
new
-
0
0
matc

new
*
450
0
new
-
0
90
new
-
303
303
match
new
-
0
0
match
new
0
new
0
new
0
new
0
new
465
new
453
new
0
new
228
new
0
new
0
new
318
new
0
new
336
new
0
new
161
new
*
129
0
new
*
0
0
match
new
*
493
0
new
0
new
137
new
373
new
479
new
404
new
0
new
0
new
0
new
396
new
383
new
0
new
0
new
267
new
0
new
0
new
0
new
204
new
0
new
131
new
443
new
0
new
0
new
0
new
0
new
*
491
248
new
*
331
0
new
0
new
-
462
462
match
new
-
498
498
match
new
-
0
471
new
0
new
0
new
0
new
479
new
404
new
65
new
0
new
0
new
396
new
0
new
0
new
314
new
59
new
0
new
0
new
0
new
0
new
317
new
0
new
291
new
0
new
478
new
134
new
0
new
-
470
470
match
new
-
437
0
new
-
0
0
match
new
462
new
0
new
363
new
300
new
419
new
0
new
0
new
479
new
0
new
330
new
0
new
0
new
0
new
457
new
0
new
*
74
443
new
*
413
413
match
new
*
390
471
new
357
new
392
new
453
new
0
new
0
new
0
new
-
254
254
match
new
*
372
0
new
*
471
0
new
-
456
456
match
new
-
0
0
match
new
-
257
257
match
new
0
new
384
new
0
new
403
new
137
new
0
new
47

In [27]:
len(recon)

300

In [45]:
dropout_cm

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
c0,214,0,152,482,479,499,162,0,0,0,...,492,0,318,162,336,0,0,*,*,*
c1,-,-,-,482,479,499,162,0,0,0,...,-,0,318,0,336,0,0,*,*,*
c2,300,386,0,39,479,500,430,0,0,0,...,-,0,*,*,456,0,257,0,384,0
c3,28,419,0,0,479,0,0,0,0,455,...,0,378,29,0,207,347,0,0,0,485
c4,0,137,0,479,404,0,0,0,396,383,...,0,0,431,0,*,*,456,462,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
c395,300,0,0,0,479,0,345,164,0,0,...,432,325,*,*,456,0,257,0,384,93
c396,450,162,0,482,479,0,-,-,-,209,...,437,0,0,210,0,0,0,180,0,0
c397,0,0,152,482,479,499,162,0,0,0,...,-,0,318,0,336,0,0,*,*,*
c398,0,435,189,-,-,-,0,0,459,470,...,0,0,291,266,0,0,0,0,0,0


In [46]:
ground_truth_cm

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,214,0,152,482,479,499,162,0,0,0,...,492,0,318,162,336,0,0,129,0,493
1,371,0,0,482,479,499,162,0,0,0,...,0,0,318,0,336,0,0,129,0,493
2,300,386,0,39,479,500,430,0,0,0,...,0,0,372,471,456,0,257,0,384,0
3,28,419,0,0,479,0,0,0,0,455,...,0,378,29,0,207,347,0,0,0,485
4,0,137,0,479,404,0,0,0,396,383,...,0,0,431,0,491,331,456,462,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,300,0,0,0,479,0,345,164,0,0,...,432,325,372,471,456,0,257,0,384,93
396,450,162,0,482,479,0,162,0,287,209,...,437,0,0,210,0,0,0,180,0,0
397,0,0,152,482,479,499,162,0,0,0,...,492,0,318,0,336,0,0,129,0,493
398,0,435,189,438,479,0,0,0,459,470,...,0,0,291,266,0,0,0,0,0,0


In [ ]:
def get_predecessor_val(tree, node, ind):
    for pred in tree.network.predecessors(node):
        print(pred.char_vec)
        if pred.char_vec[ind] == '-' or pred.char_vec[ind] == '*':
            return get_predecessor_val(tree, pred, ind)
        else:
            return pred.char_vec[ind]

In [ ]:
reconstructed_rows = []
for index, row in dropout_cm.iterrows():
    node = names_to_leaves[index]
    reconstructed_row = []
    print(row)
    for i in range(len(row)):
        print(i)
        if row[i] == '-' or row[i] == '*':
            char = get_predecessor_val(test_ppg, node, i)
            if char == None:
                char = '0'
            reconstructed_row.append(char)
            print(char)
        else:
            reconstructed_row.append(row[i])
    reconstructed_rows.append(reconstructed_row)
    print(reconstructed_row)
reconstructed_cm = pd.DataFrame(reconstructed_rows)


In [ ]:
# Iterate over the heritable dropout percentages
for recon_method in ["lookahead"]:
    print(recon_method)
    
    for herit in ["_no", ""]:
    
        for depth in range(1,9):
            print("Depth: " + str(depth))
            for drop in range(6):

                hdropout_percent = str(drop)
                print("Percent: " + hdropout_percent)

                #Establish path
                path = "/data/yosef2/users/richardz/projects/dropout_testing/heritable_testing/" + str(NUM_CELLS) + "cells/" + hdropout_percent + "percent"

                #Main loop
                accuracies = []
                for num in range(0, 50):
                    #Load the tree, the character matrix, and post-process the tree
                    test_recon = pic.load(open(path + "/" + recon_method + str(depth) + herit + "_h/dropout_cm" + str(num) + "_" + recon_method + ".pkl", 'rb'))
                    dropout_cm = pd.read_csv(path + '/dropout_cm' + str(num) + '.txt', sep = '\t', index_col = 0)
                    test_ppg = test_recon.post_process(dropout_cm)

                    #Create a dictionary to map the names of the nodes to their memory addresses
                    names_to_leaves = {}
                    for i in list(test_ppg.network.nodes):
                        if i.name != "state-node":
                            names_to_leaves[i.name] = i

                    #For each named node in the character matrix (post dropout), grab its location in memory and go through
                    #its character vector. For each missing value, grab the character from its first non-missing ancestor in 
                    #the reconstructed tree. If a character value is still missing at root, implicitly assume its value is 0.
                    #Then add the reconstructed row to the list and construct a data frame.
                    reconstructed_rows = []
                    for index, row in dropout_cm.iterrows():
                        node = names_to_leaves[index]
                        reconstructed_row = []
                        for i in range(len(row)):
                            if row[i] == "-" or row[i] == "*":
                                char = get_predecessor_val(test_ppg, node, i)
                                if char == None:
                                    char = '0'
                                reconstructed_row.append(char)
                            else:
                                reconstructed_row.append(row[i])
                        reconstructed_rows.append(reconstructed_row)
                    reconstructed_cm = pd.DataFrame(reconstructed_rows)

                    #Read in the ground-truth character matrix, before dropout is introduced
                    ground_truth_cm = pd.read_csv(path + "/ground_truth_cm" + str(num) + ".txt", sep='\t', index_col = 0)

                    #Go through the post-dropout character matrix. For each missing character in the post-dropout character matrix,
                    #check to see if the imputed character matrix matches the character in the ground-truth character matrix
                    #at that character. If yes, it is considered imputed correctly. Keep a running total of the number of total
                    #dropouts and the number of imputed correctly to create a proportion correct.
                    num_dropped = 0
                    num_correct = 0
                    for k in range(dropout_cm.shape[0]):
                        for j in range(dropout_cm.shape[1]):
                            if dropout_cm.iloc[k,j] == "-" or dropout_cm.iloc[k,j] == "*":
                                num_dropped += 1
                                if str(ground_truth_cm.iloc[k,j]) == str(reconstructed_cm.iloc[k,j]):
                                    num_correct += 1

                    accuracy = num_correct/num_dropped
                    accuracies.append(accuracy)
                    print(num, accuracy)

                #Write the proportions correctly imputed to CSV
                import csv

                with open('/data/yosef2/users/richardz/projects/dropout_testing/heritable_testing/' + str(NUM_CELLS) + 'cells/' + recon_method + str(depth) + herit + '_split_on_h_accuracies' + hdropout_percent + '.csv', 'w') as csvFile:
                    writer = csv.writer(csvFile)
                    writer.writerow(accuracies)
                csvFile.close()